In [1]:
!git clone https://github.com/aakashvardhan/custom-lit-yolov3

Cloning into 'custom-lit-yolov3'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 105 (delta 19), reused 95 (delta 9), pack-reused 0
Receiving objects: 100% (105/105), 2.42 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [2]:
%cd custom-lit-yolov3

/content/custom-lit-yolov3


In [4]:
!pip install -U lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 32.5 MB/s eta 0:00:00


In [5]:

%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/content/custom-lit-yolov3')

In [6]:
import torch
from lit_model import YOLOv3LightningModule
from lit_data_module import YOLODataModule
from pytorch_lightning import Trainer
import lightning as pl
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor, RichModelSummary,EarlyStopping
import torch
import os
import config

ImportError: ignored